# TME2 :  Appariement

L'appariement est l'action qui consiste à rassembler par paires des choses qui sont naturellement compatibles. 


In [1]:
import import_ipynb
import tme1Indexation as tm
import math
import numpy as np

importing Jupyter notebook from tme1Indexation.ipynb


# Exercice 1 – Exercice de compréhension : modèle de RI simple
On considère la collection de documents (cisi.txt et cacm.txt) et la liste des stopwords (TextRepresenter.py) du TME1. L’objectif dans cet exercice est d’estimer le score des documents pour la requˆete "home sales top"

 question 1) Quels index faut-il interroger pour avoir un calcule du score pertinent?

On utilise l'index inversé , car il nous donne accès aux informations dont on a besoin pour calculer le score plus rapidement.

question 1.2) Ecrire le code qui permet de calculer le score des documents à partir du modèle booléen

ScoreBoolean retourne les documents qui contiennent tous les termes de la requête.
Ca veut dire que j'ai considéré  que les temes de la requête sont relié par des AND entre eux.
Remarque:
si j'écris q="t1 AND (t2 OR NOT T3)", si ça c'est ma requête alors je dois retourner les documents  qui contiennent t1 et ( t2 ou t3).
Du coup ça dépend de la syntaxe de la requête.

intersection  retourne la liste des documents qui contiennent tous les mots de la requete.

In [2]:
def intersection(indexReverse,s):
    L=[]
    for c,v in s.items():
        L.append(indexReverse[c])
    d=L[0]
    M=[]
    for c,v in d.items():
        M.append(c)
    for c in M:
        b=False
        for D in L:
            if (c not in D):
                b=True
        if(b):
            M.remove(c)
    return M
    
         
    

In [3]:
def ScoreBoolean(q,T):
    I=tm.IndexerSimple()
    index,indexReverse=I.indexation(T)
    
    PS=tm.tr.PorterStemmer()
    s=PS.getTextRepresentation(q)
    return intersection(indexReverse,s)
    

On récupère la liste des documents qui contiennent tous les mots de la requête

In [4]:
q="home sales top"
T="cisi/cisi.txt"
#print(ScoreBoolean(q,T))


question 1.3) 
On calcule le score des documents à partir du modèle vectoriel (produit cart´esien) dans le cas d’une pond´eration tf

Modèle Vectoriel= Vector Space Model (VSM).

On représente les documents et les requêtes sous forme de vecteurs dans l’espace vectoriel engendré par tous les termes de la collection de documents:

T<t1,t2, …, tM>  (un terme = une dimension).

Document : dj= (w1j, w2j, …, wMj) 

Requête : q= (w1q, w2q, …, wMq) 

wij: poids du terme ti dans le document dj : tf*idf 

On parle aussi de Modèle sac de mots:

La représentation vectorielle ne tient pas compte de l’ordre des mots.

« Un garçon manque une pomme » est représenté par le même vecteur que « une pomme mange un garçon »

 c’est ce que l’on appelle « Sac de mots » (Bag of words) 
 
 Une collection de n documents et M termes distincts peut être représentée sous forme de matrice ( ligne : M document , colonnes: M terme de la collection)
 
 La requête est également représentée par un vecteur

In [5]:
#I=tm.IndexerSimple()
#index,indexReverse=I.indexation("cisi/cisi.txt")

In [6]:
#les termes de l'index (les termes normalisé présents dans la collection)
"""
T=[]
for c,v in indexReverse.items():
    T.append(c)
#La liste de tous les Documents la collection
D=[]
for c,v in index.items():
    D.append(c)

#Construction de la matrice de vecteurs de document
M={}

for d in D:
    di={} #vecteur associé à chaque document
    r=I.getTfIDFsForDoc(d)
    for w in T:
        if (w in r):
            di[w]=r[w] 
        else:
            di[w]=0
    M[d]=di
"""       

'\nT=[]\nfor c,v in indexReverse.items():\n    T.append(c)\n#La liste de tous les Documents la collection\nD=[]\nfor c,v in index.items():\n    D.append(c)\n\n#Construction de la matrice de vecteurs de document\nM={}\n\nfor d in D:\n    di={} #vecteur associé à chaque document\n    r=I.getTfIDFsForDoc(d)\n    for w in T:\n        if (w in r):\n            di[w]=r[w] \n        else:\n            di[w]=0\n    M[d]=di\n'

In [7]:
#print(M["2"])

Construction du vecteur requete. Je normalise les termes de la reqête parceque les termes de l'indexation ont tous été normalisé


In [8]:
"""
q="home sales top"
PS=tm.tr.PorterStemmer()
s=PS.getTextRepresentation(q)
Q={}
for w in T:
    if (w in s):
        Q[w]=s[w] 
    else:
        Q[w]=0
"""    

'\nq="home sales top"\nPS=tm.tr.PorterStemmer()\ns=PS.getTextRepresentation(q)\nQ={}\nfor w in T:\n    if (w in s):\n        Q[w]=s[w] \n    else:\n        Q[w]=0\n'

In [9]:
#print(Q)

La pertinence est traduite en une similarité vectorielle:
un document est d'autant plus pertinent à une requête que le vecteur associé est similaire à celui de la requête.

Le degré de correspondance de R(d,q):
Produit scalaire des deux vecteurs:
R(di,qk) = somme des produit des poids des termes de la reqête et ceux du document di , pour i=1,....,n

In [10]:
"""
Scores={}
for c,v in M.items():
    p=0
    for w in T:
        p=p+(v[w]*Q[w])
    Scores[c]=p
"""   

'\nScores={}\nfor c,v in M.items():\n    p=0\n    for w in T:\n        p=p+(v[w]*Q[w])\n    Scores[c]=p\n'

In [11]:
#print(Scores)

# Exercice 2 – Projet ”Moteur de Recherche” : Etape Appariemen

1) Représentation pondérée des documents et de la requête

Une classe générique est une classe qui peut être réutilisée pour des objets de différents types

In [16]:
class Weighter:
    
    def __init__(self,T):
        self.I=tm.IndexerSimple()
        self.index,self.indexReverse=self.I.indexation(T)
        self.mots=[c for c,v in self.indexReverse.items()] #récupère tous les mots de la collection
        self.docs=[c for c,v in self.index.items()] #répère la liste de tous les id des documents de la collection
     
     #getWeightsForDoc retourne les poids des termes pour un document dont l’identiﬁant est idDoc
        
    def getWeightsForDoc(self,idDoc):
        raise NotImplementedError
    
    
    #getWeightsForStem retourne les poids du terme stem pour tous les documents qui le contiennent
    
    def  getWeightsForStem(self,stem):
        raise NotImplementedError
        
    
    #getWeightsForQuery retourne les poids des termes de la requˆete
    
    def  getWeightsForQuery(self,query):
        raise NotImplementedError

w t,d = tf t,d et w t,q = 1 si t ∈ q, O sinon;

In [19]:
class Weighter1(Weighter):
    
    def __init__(self,T):
        self.I=tm.IndexerSimple()
        self.index,self.indexReverse=self.I.indexation(T)
        self.mots=[c for c,v in self.indexReverse.items()] #récupère tous les mots de la collection
        self.docs=[c for c,v in self.index.items()] #répère la liste de tous les id des documents de la collection
     
    def getWeightsForDoc(self,idDoc):
        d=self.index[idDoc]    
        for m in self.mots:   #tous les termes de la collection qui ne sont pas dans ce doc ont un poid de 0
            if(m not in d):
                d[m]=0
        return d
    
    def  getWeightsForStem(self,stem):
        return self.indexReverse[stem]
   
    def  getWeightsForQuery(self,query):
        PS=tm.tr.PorterStemmer()
        s=PS.getTextRepresentation(query)
        q={}
        for m in self.mots:
            if(m in s):
                q[m]=1
            else:
                q[m]=0
        return q
   

In [21]:
#w=Weighter1("cisi/cisi.txt")

Test:
on affiche le poid de tous les terme (normalisé) de la collection pour le document dont l'id est 1

In [16]:
#print(w.getWeightsForDoc("1")) 

on affiche le poid du mot "present"  "seulement" pour les documents qui le contient

In [17]:
#print(w.getWeightsForStem("present"))

on affiche pour chaque terme de la collection le poid de ce terme pour la reqête

In [18]:
#print(w.getWeightsForQuery("home saled top"))

w t,d = tf t,d et w t,q = tf t,q

In [22]:
class Weighter2(Weighter):
    
    def __init__(self,T):
        self.I=tm.IndexerSimple()
        self.index,self.indexReverse=self.I.indexation(T)
        self.mots=[c for c,v in self.indexReverse.items()] #récupère tous les mots de la collection
        self.docs=[c for c,v in self.index.items()] #répère la liste de tous les id des documents de la collection
     
    def getWeightsForDoc(self,idDoc):
        d=self.index[idDoc]    
        for m in self.mots:   #tous les termes de la collection qui ne sont pas dans ce doc ont un poid de 0
            if(m not in d):
                d[m]=0
        return d
    
    def  getWeightsForStem(self,stem):
        return self.indexReverse[stem]
   
    def  getWeightsForQuery(self,query):
        PS=tm.tr.PorterStemmer()
        s=PS.getTextRepresentation(query)
        q={}
        for m in self.mots:
            if(m in s):
                q[m]=s[m]   #il y a que cette ligne qui a changé
            else:
                q[m]=0
        return q
   

w t,d = tf t,d et w t,q = idf t si t ∈ q, 0 sinon;

In [23]:
class Weighter3(Weighter):
    
    def __init__(self,T):
        self.I=tm.IndexerSimple()
        self.index,self.indexReverse=self.I.indexation(T)
        self.mots=[c for c,v in self.indexReverse.items()] #récupère tous les mots de la collection
        self.docs=[c for c,v in self.index.items()] #répère la liste de tous les id des documents de la collection
     
    def getWeightsForDoc(self,idDoc):
        d=self.index[idDoc]    
        for m in self.mots:   #tous les termes de la collection qui ne sont pas dans ce doc ont un poid de 0
            if(m not in d):
                d[m]=0
        return d
    
    def  getWeightsForStem(self,stem):
        return self.indexReverse[stem]
   
    def  getWeightsForQuery(self,query):
        PS=tm.tr.PorterStemmer()
        s=PS.getTextRepresentation(query)
        q={}
        N=len(self.docs)
        for m in self.mots:
            if(m in s):
                idf=math.log( (1+N) / (1+len(self.indexReverse[m])) )
                q[m]=idf 
            else:
                q[m]=0
        return q

In [24]:
#w=Weighter3("cisi/cisi.txt")

In [25]:
#print(w.getWeightsForQuery("home saled top"))

 w t,d = 1 + log(tf t,d) si t ∈ d, 0 sinon; et w t,q = idf t si t ∈ q, 0 sinon

In [42]:
class Weighter4(Weighter):
    
    def __init__(self,T):
        self.I=tm.IndexerSimple()
        self.index,self.indexReverse=self.I.indexation(T)
        self.mots=[c for c,v in self.indexReverse.items()] #récupère tous les mots de la collection
        self.docs=[c for c,v in self.index.items()] #répère la liste de tous les id des documents de la collection
     
    def getWeightsForDoc(self,idDoc):
        d=self.index[idDoc]    
        di={}
        for m in self.mots:   #tous lew=Weighter3("cisi/cisi.txt")s termes de la collection qui ne sont pas dans ce doc ont un poid de 0
            if(m in d):
                di[m]=1+math.log(d[m])
            else:
                di[m]=0
        return di
    
    def  getWeightsForStem(self,stem):
        r=self.indexReverse[stem]
        d={}
        for c,v in r.items():
            d[c]=1+math.log(v)
        return d
   
    def  getWeightsForQuery(self,query):
        PS=tm.tr.PorterStemmer()
        s=PS.getTextRepresentation(query)
        q={}
        N=len(self.docs)
        for m in self.mots:
            if(m in s):
                idf=math.log( (1+N) / (1+len(self.indexReverse[m])) )
                q[m]=idf 
            else:
                q[m]=0
        return q

In [43]:
#w=Weighter4("cisi/cisi.txt")

In [29]:
#print(w.getWeightsForDoc("1")) 

In [30]:
#print(w.getWeightsForStem("present")) 

 wt,d = (1 + log(tf t,d))x idf t si t ∈ d, 0 sinon; et wt,q = (1 + log(tf t,q))x idf t si t ∈ q, 0.

In [44]:
class Weighter5(Weighter):
    
    def __init__(self,T):
        self.I=tm.IndexerSimple()
        self.index,self.indexReverse=self.I.indexation(T)
        self.mots=[c for c,v in self.indexReverse.items()] #récupère tous les mots de la collection
        self.docs=[c for c,v in self.index.items()] #répère la liste de tous les id des documents de la collection
     
    def getWeightsForDoc(self,idDoc):
        d=self.index[idDoc]    
        di={}
        N=len(self.docs)
        for m in self.mots:   #tous lew=Weighter3("cisi/cisi.txt")s termes de la collection qui ne sont pas dans ce doc ont un poid de 0
            if(m in d):
                idf=math.log( (1+N) / (1+len(self.indexReverse[m])) )
                di[m]=(1+math.log(d[m]))*idf
            else:
                di[m]=0
        return di
    
    def  getWeightsForStem(self,stem):
        r=self.indexReverse[stem]
        d={}
        N=len(self.docs)
        idf=math.log( (1+N) / (1+len(r)) )
        for c,v in r.items():           
            d[c]=(1+math.log(v))*idf
        return d
   
    def  getWeightsForQuery(self,query):
        PS=tm.tr.PorterStemmer()
        s=PS.getTextRepresentation(query)
        q={}
        N=len(self.docs)
        for m in self.mots:
            if(m in s):
                idf=math.log( (1+N) / (1+len(self.indexReverse[m])) )
                q[m]=(1+math.log(s[m]))*idf 
            else:
                q[m]=0
        return q

2) Modèles de RI

In [33]:
class IRModel:
    
    def __init__(self,T):
        self.I=tm.IndexerSimple()
        self.index,self.indexReverse=I.indexation(T)
        self.mots=[c for c,v in self.indexReverse.items()] #récupère tous les mots de la collection
        self.docs=[c for c,v in self.index.items()] #répère la liste de tous les id des documents de la collection
     
    def  getScores(self,query):   # retourne les scores des documents pour une requête
        raise NotImplementedError
        
    def getRanking(self,query): # retourne une liste de couples (document-score) ordonn´ee par score d´ecroissante
        scores=self.getScores(query)
        return sorted(scores.items(), key=lambda t: t[1],reverse=True)
    
    

question 2.2) 

Déﬁnir une classe Vectoriel qui hérite de IRModel. Elle comporte en paramètres un Weighter déﬁni dans l’´etape précédente ainsi qu’un booléen normalized permettant de déﬁnir la fonction de score (produit scalaire si faux et score cosinus si vrai). 
( les normes des vecteurs des documents ne doivent pas être calculés à chaque nouvelle requête ).


In [34]:
class Vectoriel(IRModel):
    
    def __init__(self,weighter, normalized):
        self.weighter=weighter
        self. normalized= normalized
        self.nd={}
        self.M={}
        
        for d in self.weighter.docs:
            self.M[d]=self.weighter.getWeightsForDoc(d) # pour chaque document de la collection , on récupère la liste des poids de chaque termes de la collection
   
        for c,v in self.M.items():
                d2=[]
                for ca,va in v.items():
                    d2.append(va*va)
                self.nd[c]=math.sqrt(sum(d2))  #on calcule la norme de chaque document

    def getScores(self,query):
        Scores={}
        q=self.weighter.getWeightsForQuery(query)
        for c,v in self.M.items():
                p=0              
                for w in self.weighter.mots:
                    p=p+(v[w]*q[w])
                Scores[c]=p        
                
        if(self.normalized):
            q2=[]
            for c,v in q.items():
                q2.append(v*v)
                
            nq=math.sqrt(sum(q2))
            Scores2={}
            
            for c,v in Scores.items():
                Scores2[c]=v/(self.nd[c]*nq)  
            Scores=Scores2
            
        return Scores
            

In [35]:
#V=Vectoriel(w,False)

In [36]:
#S=V.getScores("home saled top")

In [37]:
#print(S)

In [38]:
#V=Vectoriel(w,True)

In [39]:
#S=V.getScores("home saled top")

In [40]:
#print(S)
#print(V.getRanking("home saled top"))

question 2.3) Déﬁnir les classes ModeleLangue et Okapi permettant de calculer les scores pour respectivement le modèle de langue (lissage Jelinek-Mercer) et Okapi-BM25

# Modèle de langue :
Un modèle de langue est défini par son vocabulaire (mots simples, séquence de mots).
Chaque mot (m)/séquence de mots(m1m2..mn) a une probabilité d’être généré(e).
Le but est de calculer --> P(s|M):

    -s une observation (séquence de mots/texte) quelconque 
    -Probabilité d’observer s dans le modèle (la langue) M
Définir la taille des séquences générées par le modèle ? 

    --> Séquence de 1 mot, 2 mots, 3 mots, … 
 
Estimer le modèle à probabilité de chaque séquence générée ? 

Calculer la probabilité d’une observation (un texte) quelconque? 

On choisit : Séquence d’un mot --> modèle unigram .  soit s une observation (un texte) de n mots s=m1 m2…mn .

Unigram – (M génère des séquences de 1 mot) . P(S|M)= P(m1m2....mn)= produit des P(mi|M) , pour i=1,....,n.

UNigram : P(mi | M) = tf (mi) /nombre total de mots dans M.:

Problème des fréquences nulles (zéro) :

    -Si un événement (un mot de la séquence) n’apparait pas dans le modèle, le modèle lui assigne une probabilité  0 
    -Solution : assigner des probabilités différentes de zéro aux événements (mots) absents

Lissage par interpolation :

    - Les méthodes de « discounting »  traitent les mots qui n’apparaissent pas dans le corpus de la même manière. Or, il y a des mots qui peuvent être plus fréquents que d’autres 
    -Solution – Interpoler le modèle en utilisant d’autres sources d’évidence (par exemple la collection de documents) 

Interpolation  (Jelinek-Mercer) :

    -RSV(Q,d)= Produit ( (1-lambda) * p(t|Mc) + lambda*p(t|Md)), pourtous les termes t de la requête.
    P(t|Mc)=p(t)=frequence du terme t dans la collection / somme des fréquence de tous les termes de la collection
    -(λ=0.8 pour les requêtes courtes et 0.2 pour les requêtes longues) 
    


In [41]:
class ModeleLangue(IRModel):
    
    def __init__(self,weighter,l):
        self.weighter=weighter        
        L=[sum(v.values()) for c,v in self.weighter.indexReverse.items()]
        self.total=sum(L)
        self.l=l
        
    def ptD(self,t,d):
        s=sum(d.values())
        if(t in d):
            return d[t]/s
        else:
            return 0
        
    def ptC(self,t):      
        if(t in self.weighter.indexReverse):    
            return sum(self.weighter.indexReverse[t].values()) / self.total
        else:
            return 0
        
    def getScores(self,query):
        PS=tm.tr.PorterStemmer()
        s=PS.getTextRepresentation(query)
        Scores={}
        for d in self.weighter.docs:
            p=1
            for m,v in s.items():
                p=p*( (1-self.l) * self.ptC(m) + self.l*self.ptD(m,self.weighter.index[d]))
            Scores[d]=p
        return Scores

In [36]:
#M=ModeleLangue(w,0.8)

In [37]:
#print(M.getScores("home saled top"))
#print(M.getRanking("home saled top"))

# Okapi-BM25

In [38]:
class Okapi(IRModel):
    
    def __init__(self,weighter,k,b):
        self.weighter=weighter  
        self.k=k
        self.b=b

        
    def getScores(self,query):
        Scores={}
        PS=tm.tr.PorterStemmer()
        s=PS.getTextRepresentation(query)
        avgdl=0
        L=[]
        
        for c,v in self.weighter.index.items():
            t=sum(v.values())
            L.append(t)
        avgdl=(sum(L)/len(self.weighter.index))
        
        for d in self.weighter.docs:
            t=self.weighter.index[d]
            D=sum(t.values())
            for c,v in s.items():
                sc=0
                #ATTENTION : SI LE MOT DE LA REQUETE NE FIGURE PAS DANS L'INDEXINVERSE 
                if(c not in self.weighter.indexReverse ):
                    n=0
                else:
                    n=self.weighter.indexReverse[c]
                idf=math.log( (1+len(self.weighter.docs)) / (1+len(n)))
        
                if(c in t):
                    tf=t[c]
                else:
                    tf=0
                if( (tf + self.k * (1-self.b+self.b*(D/avgdl))) != 0 ):
                    sc=sc + idf * ( ( tf*(self.k+1) ) / ( tf + self.k * (1-self.b+self.b*(D/avgdl)) ) )
            Scores[d]=sc
        return Scores

In [39]:
#M=Okapi(w,1.2,0.75)

In [40]:
#print(M.getRanking("home saled top"))

# 3 Bonus - Très fortement conseillés

question 2.4)

In [41]:

Q=["convention depth  week profil","object issu assign", "inter feeder result " , 
   "emphasi length demonstrat organ" , "group receipt connect relev", "run fundament assign librari",
  "vindic howel  baxter data", "mechan timeli physical negoti", "mari historiograph desper demonstrat",
  "home saled top"]

 On sépare l’ensemble des requêtes en deux ensembles (train/test) 

In [42]:
train= Q[:int(len(Q)/2)]
test= Q[int(len(Q)/2):]

In [43]:
#print(train)
#print(test)

On déﬁnit une grille de valeurs à tester (de 0 à 1 par pas de 0.1). 

On expérimente chaque combinaison possible pour chaque modèle sur le jeu de données train

Rappel : il y a 4 classe weighter différentes , chacune a sa façon de calculer le poid d'un terme pourla requête et le poid d'un terme pour le document

# Test sur l'ensemble "train"

# Test Modele Langue

Dans le modèle de langue il y a un seul paramètre qui varie (c'est le lambda). 
(Voir en haut à quoi correspond ce lambda)


In [56]:
#w=Weighter1("cisi/cisi.txt")

Pour chaque valeur de lambda , on cré un modele de langue parametré avec ce lambda. Ensuite pour ce modèle de langue , on parcourt chaque requête dans l'ensemble train , et on calcule pour chacune de ces reqête le score associé à chaque document de la collection "cisi.txt"

In [44]:
"""
G={}
for i in np.arange(0,1.1,0.1):
    ML=ModeleLangue(w,i)
    d={}
    for c in train:
        scores=ML.getRanking(c)
        d[c]=scores
    G[i]=d
"""       

'\nG={}\nfor i in np.arange(0,1.1,0.1):\n    ML=ModeleLangue(w,i)\n    d={}\n    for c in train:\n        scores=ML.getRanking(c)\n        d[c]=scores\n    G[i]=d\n'

In [65]:
#print(G[0.1])

Observation : pour chaque valeur de lambda , on obtient pour chaque requête un ensemble de scores de documents différent

# Test Modele Vectoriel

Sur ce modèle il n'y a pas de paramètres.
Mais il peut être intéressant de changer notre weighter , car chaque weighter a sa manière de calculer le vecteur de poid des documents et de la requête

In [45]:
"""
w1=Weighter1("cisi/cisi.txt")
w2=Weighter2("cisi/cisi.txt")
w3=Weighter3("cisi/cisi.txt")
w4=Weighter4("cisi/cisi.txt")
w5=Weighter5("cisi/cisi.txt")

W=[w1,w2,w3,w4,w5]
"""

'\nw1=Weighter1("cisi/cisi.txt")\nw2=Weighter2("cisi/cisi.txt")\nw3=Weighter3("cisi/cisi.txt")\nw4=Weighter4("cisi/cisi.txt")\nw5=Weighter5("cisi/cisi.txt")\n\nW=[w1,w2,w3,w4,w5]\n'

In [46]:
"""
Gw={}
i=1
for w in W:
    V=Vectoriel(w,False)  # normalized = false  , on a utilisé la similarité produit scalaire
    d={}
    for c in train:
        scores=V.getRanking(c)
        d[c]=scores
    Gw[str(i)]=d
    i+=1
"""

'\nGw={}\ni=1\nfor w in W:\n    V=Vectoriel(w,False)  # normalized = false  , on a utilisé la similarité produit scalaire\n    d={}\n    for c in train:\n        scores=V.getRanking(c)\n        d[c]=scores\n    Gw[str(i)]=d\n    i+=1\n'

In [47]:
#print(Gw['4'])
#print(Gw['5'])

Observation : pour des weighter différents , sur chaque modele Vectoriel construit , on remarque que l'ordre des documents classé selon leur score,  obtenue sur chaque reqête , est à peu près le même

# Test Modele Okapi

Pour chaque valeur de k et b , on cré un modele Okapi parametré avec ce k et ce b. Ensuite pour ce modèle Okapi , on parcourt chaque requête dans l'ensemble train , et on calcule pour chacune de ces reqête le score associé à chaque document de la collection "cisi.txt"

In [48]:
"""
G={}
for k in np.arange(0,1.1,0.1):
    for b in np.arange(0,1.1,0.1):
        Mo=Okapi(w,k,b)
        d={}
        for c in train:
            scores=Mo.getRanking(c)
            d[c]=scores
        G[(k,b)]=d
"""       

'\nG={}\nfor k in np.arange(0,1.1,0.1):\n    for b in np.arange(0,1.1,0.1):\n        Mo=Okapi(w,k,b)\n        d={}\n        for c in train:\n            scores=Mo.getRanking(c)\n            d[c]=scores\n        G[(k,b)]=d\n'

In [49]:
#print(G[(0.0,0.0)])
#print(G[(1.0,0.8)])

On remarque que pour des valeurs de paramètres différents sur k et b , on obtient des scores de documents différents  sur chacune des requêtes, mais l'ordre de pertinence des documents sur chacune des reqêtes n'a pas l'air de changer

on pourra tester dans le TME 3 la combinaison qui obtient la meilleure valeur de métrique (MAP, Pr´ecision, ...)

on applique ces valeurs sur le jeu de test

# Test sur l'ensemble "test"

# Test Model Langue

In [50]:
"""
G={}
for i in np.arange(0,1.1,0.1):
    ML=ModeleLangue(w,i)
    d={}
    for c in test:
        scores=ML.getRanking(c)
        d[c]=scores
    G[i]=d
"""       

'\nG={}\nfor i in np.arange(0,1.1,0.1):\n    ML=ModeleLangue(w,i)\n    d={}\n    for c in test:\n        scores=ML.getRanking(c)\n        d[c]=scores\n    G[i]=d\n'

In [51]:
#print(G[0.8])
#print(G[0.9])

Observation : pour des valeurs de lambda différentes , on obtient à peu près le même ordre score de pertinence.
C'est la valeur associé à chacun des scores qui change , mais cela n'a pas l'air de modifier l'ordre de pertience des documents sur chaque requête.

# Test Model Okapi

In [52]:
"""
G={}
for k in np.arange(0,1.1,0.1):
    for b in np.arange(0,1.1,0.1):
        Mo=Okapi(w,k,b)
        d={}
        for c in test:
            scores=Mo.getRanking(c)
            d[c]=scores
        G[(k,b)]=d
"""        

'\nG={}\nfor k in np.arange(0,1.1,0.1):\n    for b in np.arange(0,1.1,0.1):\n        Mo=Okapi(w,k,b)\n        d={}\n        for c in test:\n            scores=Mo.getRanking(c)\n            d[c]=scores\n        G[(k,b)]=d\n'

In [53]:
#print(G[(0.5,0.2)])
#print(G[(0.9,0.4)])

Question 2.5) 

C'est très flou.

A la base , le but de la cross validation c'est de trouver le meilleur partionnement possible pour construire notre ensemble d'apprentissage et notre ensemble de test. Ce partionnement est important car il va nous permettre de construire le meilleur modèle possible à partir de la base d'apprentissage. Le  but étant de garder un maximum d'exemple dans notre base d'apprentissage.  Pour trouver le meilleur partionnement possible , on va diviser notre ensemble d'exemple en k folds.
(Le but étant de faire varier la valeur de k pour trouver le meilleur partionnement possible --> à la fin lensemble d'apprentissage représentera   k-1/k de l'ensemble original , et l'ensemble de test représentera  1/k de l'ensemble original).


On sépare le jeu de données en k folds et on procède à une cross validation. Chaque fold est successivement utilisé pour l’étape de test alors que les autres sont utilisés pour le train.
Pour trouver le meilleur k , lors de chaque valeurde k séléctionné,on fait la moyenne du taux d'erreur de test ( quand on choisit de prendre 1 fold pour le test et les autres pour l'entrainement , on obtient un score sur le taux d'erreur de test. La moyenne se fait sur les différents scores obtenue). 

A la fin , on récupère la valeur de k qui nous a permis d'obtenir une moyenne du taux d'erreur la plus faible.


Le Problème , c'est que dans notre cas à nous , je ne comprend pas comment le modèle apprend , car si je lui donne un ensemble de requête , il me donne le score des documents de la collection sur chaqune de ces reqêtes. Je ne vois où est la prédiciton dans tous ça.

De plus si notre objectif c'est de chercher k , alors comment mettre les bons paramètre pour constuire notre modèle.
